<a href="https://colab.research.google.com/github/Michnic120/Krzych-projects/blob/master/Module_2_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment preparation - fetching dataset and library import

In [0]:
from google.colab import drive
import os
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from scipy import misc

In [2]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [5]:
! date && unzip -q /content/drive/'My Drive'/celeb_dataset.zip -d /content/  && date #unzip dataset into local machine

Wed Nov 14 18:15:48 UTC 2018
Wed Nov 14 18:19:36 UTC 2018


In [6]:
#create a directory for preprocessed TFRecord files
!ls /content/
!mkdir /content/preprocessed/
!ls /content/

celeb  drive  sample_data
celeb  drive  preprocessed  sample_data


In [0]:
def peek_file(file_path, n_lines):
  """
  helper function to peek first n_lines of the file
  """
  try:
    n_lines = int(n_lines)
    assert(n_lines > 0)
  except ValueError:
    print("Number of lines argument must be integer-interpretable")
  except AssertionError:
    print("Number of lines must be positive")
    
  with open(file_path) as file:
    for line in range(n_lines):
      print(file.readline(), end="")
  return



# Settings

In [0]:
# path to downloaded dataset (unzipped)
DOWNLOADED_PATH = '/content/celeb' #@param {type:"string"}

# path to preprocessed dataset (tf records)
SAVED_DATASET = '/content/preprocessed/' #@param {type:"string"}

TRAIN_FILENAME = 'train.tfrecords' #@param {type:"string"}
VAL_FILENAME = 'val.tfrecords' #@param {type:"string"}
TEST_FILENAME = 'test.tfrecords' #@param {type:"string"}


# tensorflow tf records dataset settings
DECODE_PARALLEL_CALLS = 103 #@param {type:"slider", min:20, max:400, step:1}
SHUFFLE_BUFFER_SIZE = 100 #@param {type:"slider", min:20, max:400, step:1}
PREFETCH_BUFFER_SIZE = 100 #@param {type:"slider", min:20, max:400, step:1}

BATCH_SIZE = 37 #@param {type:"slider", min:1, max:1000, step:1}




In [0]:
ANNO_PATH = os.path.join(DOWNLOADED_PATH, 'Anno',  'list_attr_celeba.txt') #file that indicates to which classes an image belongs to 
IMG_PATH = os.path.join(DOWNLOADED_PATH, 'Img') #path to images directory
EVAL_PATH = os.path.join(DOWNLOADED_PATH, 'Eval', 'list_eval_partition.txt') #file that indicates how to split the dataset into training, validation and testing sets


# Decoding TFRecords

In [0]:
def dataset_wrapper(session, dataset):
    """
    Wraps TensorFlow dataset into generator.
    :param session:
    :param dataset: tf.data.Dataset object
    """
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()
    while True:
        try:
            data = session.run(next_element)
            yield data
        except tf.errors.OutOfRangeError:
            break
            
def _decode_record(record):
    """
    Decodes record from bytes to arrays.
    """
    features_dict = {
        'image': tf.FixedLenFeature((), tf.string),
        'tags': tf.FixedLenFeature((), tf.string),
    }

    features = tf.parse_single_example(record, features=features_dict)
    #data had been written as uint8 to save up space...
    image = tf.decode_raw(features['image'], tf.uint8)
    image = tf.reshape(image, shape=(218, 178, 3))
    #.. but we need more precison on calculation
    image = tf.cast(image, tf.float32)
    image = image / 255.0 #normalize
    tags = tf.decode_raw(features['tags'], tf.uint8)

    return {
        'image': image,
        'tags': tags
    }

In [0]:
# TASK 3
def make_dataset(dataset_path, batch_size=128, shuffle=False, epochs=1, count_limit=None):
    """
    Creates dataset pipeline. Use https://www.tensorflow.org/api_docs/python/tf/data/TFRecordDataset
    :param count_limit:
    :return: tf.data.TFRecordDataset object
    """
    dataset = dataset.prefetch(buffer_size=PREFETCH_BUFFER_SIZE)
    raise NotImplementedError

# Writting to TFRecords

In [0]:
peek_file(EVAL_PATH, 20)

In [0]:
#TASK 1

def _load_splits():
    """
    Load split for train/val/test dataset.
    :return:
    """
    raise NotImplementedError

In [0]:
peek_file(ANNO_PATH, 20)

In [0]:
#TASK 2

def _load_tags():
    """
    Load tags (classes) for images.
    :return:
    """
    raise NotImplementedError

In [0]:
def _bytes_feature(value):
    """
    Helper function for feature encoding.
    :param value:
    :return:
    """
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def _write_records(dataset_filename, image_filename_iterator, tags):
    """
    Writes images and tags to file.
    :param dataset_filename:
    :param image_filename_iterator: iterable containing filenames
    :param tags: dictionary containing pairs filename: tags
    :return:
    """
    with tf.python_io.TFRecordWriter(dataset_filename) as writer:
        for image_filename in tqdm(iterable=image_filename_iterator,
                                   desc='writting {} dataset'.format(dataset_filename)):
            tag = tags[image_filename]
            image = misc.imread(os.path.join(IMG_PATH, image_filename.replace('.jpg', '.png')))

            # store as uint8
            image = image.astype(np.uint8)
            feature = {
                'image': _bytes_feature([image.tostring()]),
                'tags': _bytes_feature([tag.tostring()]),
            }
            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())
    sys.stdout.flush()

In [0]:
def _create_dataset(train_filename, val_filename, test_filename):
    """
    Creates dataset in target location.
    To be implemented by students
    :param train_filename:
    :param val_filename:
    :param test_filename:
    """
    splits = _load_splits()
    tags = _load_tags()

    _write_records(train_filename, splits['0'], tags)
    _write_records(val_filename, splits['1'], tags)
    _write_records(test_filename, splits['2'], tags)

In [0]:
def maybe_create_dataset():
    """
    Function creates TensorFlow dataset if it does not exist.
    :return: (train_filename, val_filename, test_filename)
    """
    train_filename = os.path.join(SAVED_DATASET, TRAIN_FILENAME)
    val_filename = os.path.join(SAVED_DATASET, VAL_FILENAME)
    test_filename = os.path.join(SAVED_DATASET, TEST_FILENAME)

    if not all([os.path.exists(filename)
                for filename
                in [train_filename, val_filename, test_filename]]):
        _create_dataset(train_filename, val_filename, test_filename)
    return train_filename, val_filename, test_filename

#Main pipeline

In [0]:
train_filename, val_filename, test_filename = maybe_create_dataset()

In [0]:
with tf.Session() as session:
    train_set = make_dataset(
        dataset_path=test_filename,
        batch_size=BATCH_SIZE,
        shuffle=True,
        epochs=5,
        count_limit=COUNT_LIMIT,
    )

In [0]:
with tf.Session() as session:    
    for data in tqdm(dataset_wrapper(session, train_set)):
        break
        

    for data in tqdm(dataset_wrapper(session, train_set)):
        plt.imshow(data['image'][0])
        plt.show()
        break